In [34]:
from TM1py import TM1Service,Element
import logging
import configparser
import argparse
logging.basicConfig(filename='BU_Diff.txt',level=logging.INFO,format='%(asctime)s -%(name)s -%(levelname)s - %(message)s')

In [46]:
config=configparser.ConfigParser()
config.read(r'C:\Users\User\Desktop\config.ini')

['C:\\Users\\User\\Desktop\\config.ini']

In [48]:
tm1=TM1Service(**config['Neil'])

In [71]:
p=tm1.processes.get(name_process='99A.Rest Api Test Sleep For 30s')

In [75]:
process='process1'
success,status,error_log=tm1.processes.execute_with_return(process_name=process)

In [80]:
def diff_bu(tm1:TM1Service,dim_name:str,dim_name_compare:str,ele_name:str):
    hier=tm1.hierarchies.get(dim_name,dim_name)
    hier_compare=tm1.hierarchies.get(dim_name_compare,dim_name_compare)
    desc=hier.get_descendants(element_name=ele_name,recursive=True)
    desc_compare=hier_compare.get_descendants(element_name=ele_name,recursive=True)
    diff=desc_compare-desc
    if diff != set():
        #日志输出差异
        logging.info(dim_name_compare+' new updated bu dimension generates diff:'+str(diff))
    else:
        logging.info(dim_name_compare+' new updated bu dimension validated as ok')
        

In [79]:
def update_and_validate_bu(tm1:TM1Service,active_flag:str,dim_name:str,dim_name_compare:str,ele_name:str):
    if active_flag=='N':
        #构建validation bu维度
        process='process1'
        success,status,error_log=tm1.processes.execute_with_return(process_name=process)
        #输出PA脚本执行异常日志
        if not scuccess:
            content=tm1.processes.get_error_log_file_content(error_log)
            logging.info(content)
        #输出差异
        diff_bu(tm1,dim_name,dim_name_compare,ele_name)
    if active_flag=='Y':
        #构建business unit fpa维度
        process='process2'
        success,status,error_log=tm1.processes.execute_with_return(process_name=process)
        #输出PA脚本执行异常日志
        if not scuccess:
            content=tm1.processes.get_error_log_file_content(error_log)
            logging.info(content)
        #输出差异
        diff_bu(tm1,dim_name,dim_name_compare,ele_name)
    

In [70]:
diff_bu(tm1,dim_name='Account PL',dim_name_compare='Account PL Clone',ele_name='PL100 - 三、营业利润')